In [ ]:
!pip install python-binance
import pandas as pd
import sqlalchemy
from binance.client import Client
from binance import BinanceSocketManager

In [ ]:
api_key = "abc"
api_secret = "xyz"

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')

In [ ]:
df = pd.read_sql('BTCUSDT',engine)

In [ ]:
df.Price.plot()

In [ ]:
#trend following
#if the crypto was rising by x% -> Buy
#exit when profit is above 0.15% or loss is crossing -0.15%

In [ ]:
def strategy(entry, lookback, qty, open_position=False):
    while True:
          df = pd.read_sql('BTCUSDT',engine)
          lookbackperiod = df.iloc[-lookback:]
          cumret = (lookbackperiod.Price.pct_change() +1).cumprod() - 1
          if not open_position:
             if cumret[cumret.last_valid_index()] > entry:
                order = client.create_order(symbol=pair,
                                           side='BUY',
                                           type='MARKET',
                                           quantity=qty)
                print(order)
                open_position = True
                break
    if open_position:
        while True:
            df = pd.read_sql('BTCUSDT', engine)
            sincebuy = df.loc[df.Time > 
                              pd.to_datetime(order['transactTime'],
                                            unit='ms')]
            if len(sincebuy) > 1:
                sincebuyret = (sincebuy.Price.pct_change() +1).cumprod() - 1
                last_entry = sincebuyret[sincebuyret.last_valid_index()]
                if last_entry > 0.0015 or last_entry < -0.0015:
                    order = client.create_order(symbol=pair,
                                           side='SELL',
                                           type='MARKET',
                                           quantity=qty)
                    print(order)
                    break      